In [2]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load intents data from JSON file
with open('/content/Intents.json') as file:
    intents_data = json.load(file)

# Extract data from intents
patterns = []
responses = []
tags = []
for intent in intents_data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern.lower())  # Convert to lowercase for consistency
        responses.append(intent['responses'][0])  # Assuming only one response per pattern
        tags.append(intent['tag'])

# Tokenize input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)

vocab_size = len(tokenizer.word_index) + 1
max_len = max([len(seq) for seq in tokenizer.texts_to_sequences(patterns)])

# Convert text data into sequences
pattern_sequences = tokenizer.texts_to_sequences(patterns)

# Pad sequences to make them of equal length
pattern_pad = pad_sequences(pattern_sequences, maxlen=max_len, padding='post')

# Convert tags to categorical labels
tag_labels = np.array(tags)
label_indices = {tag: i for i, tag in enumerate(set(tags))}
tag_labels = [label_indices[tag] for tag in tag_labels]

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
    LSTM(64),
    Dense(len(label_indices), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(pattern_pad, np.array(tag_labels), epochs=100, verbose=2)

# Function to generate a response
# Function to generate a response
def generate_response(user_input):
    user_input_seq = tokenizer.texts_to_sequences([user_input.lower()])
    user_input_pad = pad_sequences(user_input_seq, maxlen=max_len, padding='post')
    predicted_probs = model.predict(user_input_pad)[0]
    predicted_tag = np.argmax(predicted_probs)
    predicted_tag_name = [tag for tag, index in label_indices.items() if index == predicted_tag][0]
    for intent in intents_data['intents']:
        if intent['tag'] == predicted_tag_name:
            return np.random.choice(intent['responses'])


Epoch 1/100
2/2 - 2s - loss: 2.9972 - accuracy: 0.0500 - 2s/epoch - 1s/step
Epoch 2/100
2/2 - 0s - loss: 2.9933 - accuracy: 0.0500 - 21ms/epoch - 10ms/step
Epoch 3/100
2/2 - 0s - loss: 2.9905 - accuracy: 0.0500 - 18ms/epoch - 9ms/step
Epoch 4/100
2/2 - 0s - loss: 2.9873 - accuracy: 0.0500 - 21ms/epoch - 11ms/step
Epoch 5/100
2/2 - 0s - loss: 2.9845 - accuracy: 0.0667 - 18ms/epoch - 9ms/step
Epoch 6/100
2/2 - 0s - loss: 2.9814 - accuracy: 0.1167 - 21ms/epoch - 10ms/step
Epoch 7/100
2/2 - 0s - loss: 2.9770 - accuracy: 0.1500 - 18ms/epoch - 9ms/step
Epoch 8/100
2/2 - 0s - loss: 2.9731 - accuracy: 0.1500 - 18ms/epoch - 9ms/step
Epoch 9/100
2/2 - 0s - loss: 2.9685 - accuracy: 0.1500 - 18ms/epoch - 9ms/step
Epoch 10/100
2/2 - 0s - loss: 2.9629 - accuracy: 0.1167 - 20ms/epoch - 10ms/step
Epoch 11/100
2/2 - 0s - loss: 2.9558 - accuracy: 0.1167 - 19ms/epoch - 9ms/step
Epoch 12/100
2/2 - 0s - loss: 2.9479 - accuracy: 0.1167 - 19ms/epoch - 9ms/step
Epoch 13/100
2/2 - 0s - loss: 2.9374 - accuracy:

In [4]:
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input)
    print("Bot:", response)

You: Hi
1/1 [==============================] - 0s 26ms/step
Bot: Hello! Welcome to our restaurant. How can I assist you today?
You: What are your opening hours?
1/1 [==============================] - 0s 37ms/step
Bot: Our opening hours are 9am to 10pm . We're currently open.
You: I'd like to make a reservation
1/1 [==============================] - 0s 26ms/step
Bot: Certainly! How many people will be in your party and for what time?
You: There will be 5 of us
1/1 [==============================] - 0s 22ms/step
Bot: Great! And what time would you like to make the reservation for?
You: Evening at 6 o'clock
1/1 [==============================] - 0s 25ms/step
Bot: Let me check for availability...
You: Is a table available?
1/1 [==============================] - 0s 24ms/step
Bot: Yes, a table is available for 5 people at 6 o'clock in the evening.
You: How can I pay?
1/1 [==============================] - 0s 23ms/step
Bot: We accept cash, credit/debit cards, and mobile payments like Apple Pa